In [ ]:
import os
import sys

import matplotlib.pyplot as plt
import pandas as pd
import requests


PROJECT_ROOT = os.path.abspath(os.path.join(os.getcwd(), '..'))
if PROJECT_ROOT not in sys.path:
    sys.path.append(PROJECT_ROOT)

from src.ingestion.auth import strava_auth  # noqa: E402

In [ ]:
athlete_url = 'https://www.strava.com/api/v3/athlete'
access_token = strava_auth.get_access_token()
headers = {'Authorization': f'Bearer {access_token}'}

# General athlete information

In [ ]:
athlete_url = 'https://www.strava.com/api/v3/athlete'
response = requests.get(athlete_url, headers=headers, timeout=10)
response.raise_for_status()
data = response.json()
data

In [ ]:
data

In [ ]:
athlete_id = data['id']
print(f'Athlete ID: {athlete_id}')

In [ ]:
# Method using the strava extractor class
from src.ingestion.extractors.strava_extractor import StravaExtractor


extractor = StravaExtractor(access_token)
athlete_info = extractor.fetch_athlete_info()
athlete_info

In [ ]:
athlete_info.id

In [ ]:
df_athlete = pd.DataFrame([athlete_info.model_dump()])
df_athlete

# Athlete stats

In [ ]:
athlete_stats_url = f'https://www.strava.com/api/v3/athletes/{athlete_id}/stats'
r = requests.get(athlete_stats_url, headers=headers, timeout=10)
r.raise_for_status()

In [ ]:
athlete_stats = r.json()
athlete_stats

In [ ]:
df_athlete_stats = pd.DataFrame([athlete_stats])
df_athlete_stats

In [ ]:
from src.ingestion.extractors.strava_extractor import StravaExtractor


extractor = StravaExtractor(access_token)
athlete_stats = extractor.fetch_athlete_stats(athlete_id)

In [ ]:
athlete_stats

In [ ]:
df_athlete_stats_new = pd.DataFrame([athlete_stats.model_dump()])
df_athlete_stats_new

# Gears

- Iterate over all activities and get unique gear ids

In [ ]:
gear_id = 'g20984891'
gear_2 = 'b16370167'

In [ ]:
gear_url = f'https://www.strava.com/api/v3/gear/{gear_2}'
r = requests.get(gear_url, headers=headers, timeout=10)
gear = r.json()
gear

# Activity Details

## Streams

In [ ]:
activity_id = '16749652500'  # '16749652500' # '16222086377'

In [ ]:
stream_url = f'https://www.strava.com/api/v3/activities/{activity_id}/streams?heartrate'
r = requests.get(stream_url, headers=headers, timeout=10)
stream = r.json()

In [ ]:
stream

In [ ]:
stream_url = f'https://www.strava.com/api/v3/activities/{activity_id}/streams'

params = {
    'keys': 'time,distance,latlng,altitude,velocity_smooth,heartrate,cadence,watts,temp,moving,grade_smooth',  # or use 'types'
    'key_by_type': 'true',
}

r = requests.get(stream_url, headers=headers, params=params, timeout=10)
r.raise_for_status()
stream = r.json()

print(
    stream.keys()
)  # e.g. dict_keys(['time', 'heartrate', 'altitude', 'velocity_smooth'])

In [ ]:
stream['grade_smooth']

In [ ]:
df_stream = pd.DataFrame(stream)
df_stream

In [ ]:
stream_url = f'https://www.strava.com/api/v3/activities/{activity_id}/streams'

params = {
    'keys': 'time,distance,latlng,altitude,velocity_smooth,heartrate,cadence,watts,temp,moving,grade_smooth',  # or use 'types'
    'key_by_type': 'true',
}

r = requests.get(stream_url, headers=headers, params=params, timeout=10)
r.raise_for_status()
stream = r.json()

# --- Convert to DataFrame ---
df = pd.DataFrame({
    'time_s': stream['time']['data'],
    'heart_rate': stream['grade_smooth']['data'],
})

# Optional: convert seconds to minutes
df['time_min'] = df['time_s'] / 60

# --- Plot ---
plt.figure(figsize=(10, 5))
plt.plot(df['time_min'], df['heart_rate'], linewidth=1.5)
plt.title(f'Heart Rate Stream for Activity {activity_id}')
plt.xlabel('Time (minutes)')
plt.ylabel('Heart Rate (bpm)')
plt.grid(True)
plt.tight_layout()
plt.show()

## Comments

In [ ]:
comment_url = f'https://www.strava.com/api/v3/activities/{activity_id}/comments'
r = requests.get(comment_url, headers=headers, timeout=10)
r.raise_for_status()
comments = r.json()
comments

## Kudos

In [ ]:
kudos_url = f'https://www.strava.com/api/v3/activities/{activity_id}/kudos'
r = requests.get(kudos_url, headers=headers, timeout=10)
r.raise_for_status()
kudos = r.json()
kudos

## Laps

In [ ]:
laps_url = f'https://www.strava.com/api/v3/activities/{activity_id}/laps'
r = requests.get(laps_url, headers=headers, timeout=10)
r.raise_for_status()
laps = r.json()
laps

## Zones

In [ ]:
zones_url = f'https://www.strava.com/api/v3/activities/{activity_id}/zones'
r = requests.get(zones_url, headers=headers, timeout=10)
r.raise_for_status()
zones = r.json()
zones